## Bibliotecas

In [7]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.window import Window

In [8]:
import findspark

findspark.init()

In [9]:
spark = (
    SparkSession.builder
    .config('spark.serializer', "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

In [10]:
imdb_path = '../data/imdb/'

In [11]:
df_titles = spark.read.format("parquet").load(imdb_path + 'title_basics')
df_ratings = spark.read.format("parquet").load(imdb_path + 'title_ratings')

## Questão 11

### Forma Errada

In [16]:
(
    df_titles.withColumn('genres', f.explode(f.split('genres', ',')))
    .filter("startYear = 2018")
    .groupby('startYear', 'genres')
    .count()
    .withColumn('perc', f.col('count')/f.sum('count').over(Window.partitionBy('startYear')))
    .filter('genres = "Comedy"')
    .toPandas()
)

,startYear,genres,count,perc
0,2018,Comedy,78809,0.126122


### Forma Correta

In [13]:
(
    df_titles
    .withColumn('n_titles', f.count(f.lit(1)).over(Window.partitionBy('startYear')))
    .withColumn('genres', f.explode(f.split('genres', ',')))
    .filter("startYear = 2018")
    .groupby('startYear', 'genres', 'n_titles')
    .count()
    .withColumn('perc', f.col('count')/f.col('n_titles'))
    .filter('genres = "Comedy"')
    .toPandas()
)

,startYear,genres,n_titles,count,perc
0,2018,Comedy,402244,78809,0.195923
